# Urban Informatics
# Module 06: geopandas

- introduce the Python geospatial data science stack
- working with spatial data
- geometric operations
- projection
- mapping
- spatial joins
- spatial indexing

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from shapely.geometry import Point

%matplotlib notebook

## 1. Quick overview of mapping and projection concepts

### Some terminology:

- **geoid**: (that's *gee-oid*) the surface of the earth's gravity field, which approximates mean sea level
- **spheroid** or **ellipsoid** (interchangeable terms): a model that smoothly approximates the geoid
- **datum**: based on spheroid but incorporates local variations in the shape of the Earth. Used to describe a point on the Earth's surface, such as in latitude and longitude.
  - NAD83 (North American Datum 1983) uses the GRS80 spheroid
  - WGS84 (World Geodetic Survey 1984 datum) uses the WGS84 spheroid
  - The latitude and longitude coordinates of some point differ slightly based on the datum. GPS uses WGS84.
- **coordinate reference system** (CRS) or spatial reference system (SRS): a series of parameters that [define](http://spatialreference.org/) the coordinate system and spatial extent (aka, domain) of some dataset.
- **geographic coordinate system** (GCS): specifies a datum, spheroid, units of measure (such as meters), and a prime meridian
- **projected coordinate system** or map projection: projects a map of the Earth's 3-D spherical surface onto a flat surface that can be measured in units like meters. Here's a [list of projections](https://en.wikipedia.org/wiki/List_of_map_projections).
- **eastings** and **northings**: the x and y coordinates of a projected map, usually measured in meters
- **false origin**: the 0,0 origin point from which eastings and northings are measured on the map, usually the lower left corner rather than the center
- **PROJ.4**: a library to convert/project spatial data with consistent CRS [parameter names](https://github.com/OSGeo/proj.4/wiki/GenParms)

### Common CRS parameters (and their PROJ.4 names):

- datum (datum)
- ellipse (ellps)
- projection (proj)
  - the name of the projected coordinate system, such as Albers Equal Area (aea) or Lambert Conformal Conic (lcc)
- standard parallels (lat_1, lat_2)
  - where the projection surface touches the globe - at the standard parallels, the projection shows no distortion
- central meridian and latitude of origin (lon_0, lat_0)
  - the origin of the projection's x and y coordinates (eastings and northings) - usually the center of the map projection
- false easting and false northing (x_0, y_0)
  - offsets to add to all your eastings and northings - usually used to make all the coordinates on the map positive numbers by starting 0,0 at the lower left corner rather than the center of the map (see false origin, above)

### Common projection types:

- *equal area* projections: maintain area at the expense of shape, distance, and direction - such as the [Albers Equal Area](https://en.wikipedia.org/wiki/Albers_projection) projection
- *conformal* projections: maintain shapes at the expense of area, distance, and direction - such as the [Lambert Conformal Conic](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) projection
- *equidistant* projections: [preserve distance](https://en.wikipedia.org/wiki/Map_projection#Equidistant) from one point or along all meridians and parallels
- *azimuthal* projections: maintain direction from one point to all other points - such as an [orthographic](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) projection
- others compromise to minimize overall distortion or aim for aesthetic value - such as the [Robinson](https://en.wikipedia.org/wiki/Robinson_projection) projection

## 2. Working with spatial data

We'll use [geopandas](http://geopandas.org/user.html), which spatializes pandas dataframes. If you're loading a CSV file of lat-lng data, first load it the usual way with pandas, then create a new geopandas GeoDataFrame from your DataFrame.

In [ ]:
# load usa point data as a regular pandas dataframe
df = pd.read_csv('data/usa-latlong.csv')
df.shape

In [ ]:
# convert it to a geopandas geodataframe
usa_points = gpd.GeoDataFrame(df)
usa_points.head()

In [ ]:
# measure distance between the first two points in the dataset using geopy's great_circle function
# notice the points are represented as (lat, long) to do the distance calculation
# (lat, long) is equivalent to (y, x) rather than (x, y)
point0 = usa_points.loc[0, 'latitude'], usa_points.loc[0, 'longitude']
point1 = usa_points.loc[1, 'latitude'], usa_points.loc[1, 'longitude']
great_circle(point0, point1).miles

In [ ]:
# how far is each row from the memorial coliseum?
def get_distance(row):
    memorial_coliseum = 34.014033, -118.287868
    distance = great_circle(memorial_coliseum, (row['latitude'], row['longitude'])).miles
    return round(distance, 2)

usa_points['miles_from_coliseum'] = usa_points.apply(get_distance, axis=1)
usa_points.head()

In [ ]:
# now it's your turn
# 1. how far is the coliseum from your home (look up its coordinates on google maps)?
# 2. how far is LA central library from griffith observatory?


In [ ]:
# load the states shapefile as a geodataframe
states = gpd.GeoDataFrame.from_file('data/states_21basic/states.shp')
len(states)

In [ ]:
states.head()

In [ ]:
# what's in our shapefile?
states['STATE_NAME'].sort_values().unique()

In [ ]:
# or read in a geojson file
states2 = gpd.read_file('data/states.geojson')
states2.head()

In [ ]:
# you can edit your data then save the gdf to disk
states2['STATE_NAME'] = states2['STATE_NAME'].str.upper()
states2.to_file('data/states2')

In [ ]:
# now it's your turn
# which SUB_REGION contains the most states? and the fewest?


## 2. Geometric operations

In [ ]:
# create a geometry column in our point dataset to contain shapely geometry for geopandas to use
# notice the points are represented as long, lat so that they are equivalent to x, y
usa_points['geometry'] = usa_points.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
usa_points.head()

### 2.1. intersects

In [ ]:
# get those states that intersect with the point data
# use geopandas unary_union attribute to get a single geometry object representing all the points
mask = states['geometry'].intersects(usa_points['geometry'].unary_union)
len(states[mask])

In [ ]:
# which states didn't have any point data?
states[~mask]['STATE_NAME']

### 2.2. within

In [ ]:
# get the geometry of the state with name california, as a geoseries of one element
california = states[states['STATE_NAME']=='California']['geometry']

# use iloc to extract the value from a geoseries of one element, to a shapely polygon
california_polygon = california.iloc[0]
california_polygon

In [ ]:
# find all the points in CA
cal_points = usa_points[usa_points.within(california_polygon)]
len(cal_points)

In [ ]:
fig, ax = plt.subplots()
ax = california.plot(ax=ax, color='#999999')
ax = cal_points.plot(ax=ax, c='r', markersize=1)
ax.axis('off')
plt.show()

In [ ]:
# another example: remove any point data that lies within Alaska or Hawaii
# first grab the polygons from these states' geoseries using iloc
alaska_polygon = states[states['STATE_NAME']=='Alaska']['geometry'].iloc[0]
hawaii_polygon = states[states['STATE_NAME']=='Hawaii']['geometry'].iloc[0]

# now use a mask to select all points *not* within either state's polygon
alaska_hawaii_mask = usa_points.within(alaska_polygon) | usa_points.within(hawaii_polygon)
contiguous_usa_points = usa_points[~alaska_hawaii_mask]
len(contiguous_usa_points)

In [ ]:
# now it's your turn
# how many points are in arizona?
# how many are in a "pacific" state but not california?


### 2.3 dissolve to aggregate

In [ ]:
mask = (states['STATE_NAME']!='Alaska') & (states['STATE_NAME']!='Hawaii')
contiguous_states = states[mask]

In [ ]:
subregions = contiguous_states.dissolve('SUB_REGION')
ax = subregions.plot(linewidth=1, edgecolor='w')

### 2.4 convex hulls, envelopes, and centroids

In [ ]:
texas = states[states['STATE_NAME']=='Texas']
ax = texas.convex_hull.plot(color='b')
ax = texas.plot(ax=ax, color='pink')

In [ ]:
ax = texas.envelope.plot(color='gray')
ax = texas.convex_hull.plot(ax=ax, color='b')
ax = texas.plot(ax=ax, color='pink')

In [ ]:
ax = texas.envelope.plot(color='gray')
ax = texas.convex_hull.plot(ax=ax, color='b')
ax = texas.plot(ax=ax, color='pink')
ax = texas.centroid.plot(ax=ax, color='r')

In [ ]:
ax = contiguous_states.plot(color='gray', linewidth=0.5, edgecolor='w')
ax = contiguous_states.centroid.plot(ax=ax, color='r', markersize=5)

In [ ]:
# now it's your turn
# calculate the centroids of california's state boundary, convex hull, and envelope
# how far is california's centroid from oregon's centroid?


### 2.5. buffer, difference, and intersection

In [ ]:
# you can easily calculate buffers
california_polygon.buffer(distance=1)

In [ ]:
buffered_points = cal_points.buffer(0.5)
ax = california.difference(buffered_points.unary_union).plot()

In [ ]:
ax = california.intersection(buffered_points.unary_union).plot()

But these buffers are weird because the data is not projected. It's all in lat-long degrees. Let's project it.

## 3. Projection

Let's define a projection we can use to convert and map our lat-long data. The parameters in the following dictionaries correspond to the projection parameters from PROJ4. Geopandas uses the pyproj library to project spatial data, which in turn uses PROJ4 projection names and parameters.

You can figure out these parameter values either by approximating the lats and longs of your spatial data set, or by trial and error, or by looking up a reference like [this one](http://spatialreference.org/ref/epsg/26911/) for UTM zone 11. 

In [ ]:
# you must specify the geodataframe's original CRS (if it doesn't already have one) so geopandas knows how to project it
# the GPS data is lat-long and its datum/ellipsoid is WGS84 - this is a geographic coordinate system
original_crs = {'init' : 'epsg:4326'}
cal_points.crs = original_crs
cal_points.crs

In [ ]:
california.crs

In [ ]:
ax = california.plot()

In [ ]:
# specify your projection manually
# we'll map with UTM zone 11 which is good for California - this is a projected coordinate system
utm_11 = '+proj=utm +zone=11 +ellps=GRS80 +datum=NAD83 +units=m +no_defs'

In [ ]:
# the california shapefile already has an original CRS so we don't need to specify it - just project and plot it
california = california.to_crs(utm_11)
ax = california.plot()

**Be careful**: heed the difference between `gdf.crs` and `gdf.to_crs()`

In [ ]:
# now project the point data to the projected coordinate system, draw 30km buffers, and plot it
cal_points = cal_points.to_crs(utm_11)
ax = california.plot(color='gray')
ax = cal_points.buffer(30000).plot(ax=ax, color='none', edgecolor='r')

So that's our projected data and shapefile. Notice how the shape has changed, and how the units make more sense - they are in meters now. So our buffers are a 20km radius from each point.

In [ ]:
# now it's your turn
# find all the points that intersect with massachusetts
# project the points and the state boundary to a new projection of your choosing
# plot them together, like you saw above


Now let's project our entire USA points data to a projection appropriate for the entire USA. We'll specify the datum, ellipsoid, projection name, standard parallels, central meridian and latitude of origin, false easting and false northing (because matplotlib basemap sticks the origin at the lower left corner), and measurement units. 

In [ ]:
# set our usa_points dataset's original CRS
usa_points.crs = original_crs
usa_points.head()

In [ ]:
# specify the width and height of the map extent/domain in projection coordinate units (meters) - approx USA dimensions
map_width_m = 5000 * 1000 #5000 km
map_height_m = 3500 * 1000 #3500 km

In [ ]:
# Albers Conical Equal Area projection for USA
albers_usa = {'datum':'NAD83',
              'ellps':'GRS80',
              'proj':'aea', 
              'lat_1':33, 
              'lat_2':45, 
              'lon_0':-97, 
              'lat_0':39, 
              'x_0':map_width_m/2, 
              'y_0':map_height_m/2,
              'units':'m'}

In [ ]:
# now you can convert the point data to a projected CRS
# doesn't save the changes unless you do assignment
usa_points.to_crs(albers_usa).head()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 8))
contiguous_states.plot(ax=axes[0])
contiguous_states.to_crs(albers_usa).plot(ax=axes[1])
plt.show()

Unprojected lat-long data (left) and projected data (right). The origin on the right is 0,0 like we'd expect for our false origin.

In [ ]:
# states by area (square degrees???)
contiguous_states = contiguous_states.copy() #bc we sliced this from states gdf earlier
contiguous_states['area'] = contiguous_states.area
contiguous_states.sort_values('area', ascending=False).head()[['STATE_NAME', 'area']]

In [ ]:
# states by area (sq km!!!)
contiguous_states_proj = contiguous_states.to_crs(albers_usa)
contiguous_states_proj['area'] = contiguous_states_proj.area / 1e9 # sq meters -> sq km
contiguous_states_proj.sort_values('area', ascending=False).head()[['STATE_NAME', 'area']]

In [ ]:
# now it's your turn
# dissolve the states into their regions. which region is the smallest? what is its area in square km?


## 4. Simple Mapping

In [ ]:
ax = contiguous_states_proj.plot(column='area', cmap='viridis', scheme='quantiles',
                                 edgecolor='k', linewidth=0.25)

In [ ]:
fig, ax = plt.subplots()
ax = contiguous_states_proj.plot(ax=ax, column='area', cmap='viridis', scheme='quantiles',
                                 edgecolor='k', linewidth=0.25)
ax.axis('off')
fig.savefig('data/map.png', dpi=600)
plt.show()

We'll be doing more mapping, and interactive mapping, next week.

## 5. Spatial joins and spatial indexes

In [ ]:
rents = pd.read_csv('data/listings.csv')
len(rents)

In [ ]:
rents = gpd.GeoDataFrame(rents)
rents['geometry'] = rents.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
rents.crs = original_crs
rents.head()

In [ ]:
rents.crs == contiguous_states.crs

In [ ]:
rents = rents.to_crs(contiguous_states.crs)

In [ ]:
rents_states = gpd.sjoin(rents, contiguous_states, how='left', op='within')

In [ ]:
groups = rents_states.groupby('STATE_NAME')

In [ ]:
# which states have the highest median rents?
groups['rent'].median().sort_values(ascending=False).head()

In [ ]:
# which states have the most bedrooms/unit in the listings?
groups['bedrooms'].mean().sort_values(ascending=False).head()

### spatial indexing

In [ ]:
geometry = states[states['STATE_NAME']=='Massachusetts']['geometry'].iloc[0]
geometry

In [ ]:
%%time
matches = rents.intersects(geometry)

In [ ]:
sindex = rents.sindex

In [ ]:
%%time
possible_matches_index = list(sindex.intersection(geometry.bounds))
possible_matches = rents.iloc[possible_matches_index]
precise_matches = possible_matches[possible_matches.intersects(geometry)]

## In-class workshop

1. Download census tract boundaries and county boundaries for some state from [TIGER](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)
2. Load them with geopandas
3. Create a choropleth map, colored by some attribute in the shapefile
4. Spatial join the rental listings to the tracts
5. Which tract in your state has the highest median rent?
6. Spatial join tracts to counties
7. Dissolve your tracts into counties
8. Which county in your state has the highest median rent?